In [ ]:
!pip install -q "transformers>=4.44.0" datasets sacrebleu evaluate accelerate sentencepiece


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


ValueError: mount failed

In [ ]:
BASE_DIR = "/content/drive/MyDrive/vlsp_mt_share"

TRAIN_EN_PATH    = f"{BASE_DIR}/train.en.txt"
TRAIN_VI_PATH    = f"{BASE_DIR}/train.vi.txt"
TRAIN_TSV        = f"{BASE_DIR}/train_envi.tsv"
TRAIN_SPLIT_TSV  = f"{BASE_DIR}/train_envi_train.tsv"
DEV_SPLIT_TSV    = f"{BASE_DIR}/train_envi_dev.tsv"

MODEL_DIR        = f"{BASE_DIR}/opus_envi_vlsp_best"
PUBLIC_TEST_EN   = f"{BASE_DIR}/public_test.en.txt"
PUBLIC_TEST_VI   = f"{BASE_DIR}/public_test.vi.txt"
RESULTS_TSV      = f"{BASE_DIR}/public_test_results_en_vi.tsv"


In [ ]:
with open(TRAIN_EN_PATH, encoding="utf-8") as f:
    en_lines = [l.strip() for l in f.readlines()]
with open(TRAIN_VI_PATH, encoding="utf-8") as f:
    vi_lines = [l.strip() for l in f.readlines()]


In [ ]:
en_path = "/content/drive/MyDrive/vlsp_mt_share/train.en.txt"   # sửa nếu path khác
vi_path = "/content/drive/MyDrive/vlsp_mt_share/train.vi.txt"   # path em đã có

with open(en_path, encoding="utf-8") as f:
    en_lines = [l.strip() for l in f.readlines()]

with open(vi_path, encoding="utf-8") as f:
    vi_lines = [l.strip() for l in f.readlines()]

print("Số dòng EN:", len(en_lines))
print("Số dòng VI:", len(vi_lines))
print("EN mẫu:", en_lines[0])
print("VI mẫu:", vi_lines[0])


In [ ]:
!pip install -q "transformers==4.57.3" datasets sacrebleu evaluate sentencepiece


In [ ]:
import pandas as pd

with open(TRAIN_EN_PATH, encoding="utf-8") as f:
    en_lines = [l.strip() for l in f.readlines()]

with open(TRAIN_VI_PATH, encoding="utf-8") as f:
    vi_lines = [l.strip() for l in f.readlines()]

print("Số dòng EN:", len(en_lines))
print("Số dòng VI:", len(vi_lines))
print("EN mẫu:", en_lines[0])
print("VI mẫu:", vi_lines[0])

df = pd.DataFrame({
    "src": en_lines,
    "tgt": vi_lines,
})

df.to_csv(TRAIN_TSV, sep="\t", index=False)
print("Đã lưu:", TRAIN_TSV, "– số dòng:", len(df))


In [ ]:
from sklearn.model_selection import train_test_split

df = pd.read_csv(TRAIN_TSV, sep="\t")

train_df, dev_df = train_test_split(df, test_size=0.05, random_state=42)

train_df.to_csv(TRAIN_SPLIT_TSV, sep="\t", index=False)
dev_df.to_csv(DEV_SPLIT_TSV, sep="\t", index=False)

print("Train:", len(train_df), "Dev:", len(dev_df))
print("Train TSV:", TRAIN_SPLIT_TSV)
print("Dev TSV:", DEV_SPLIT_TSV)


Train: 475000 Dev: 25000
Train TSV: /content/drive/MyDrive/vlsp_mt_share/train_envi_train.tsv
Dev TSV: /content/drive/MyDrive/vlsp_mt_share/train_envi_dev.tsv


In [ ]:
from datasets import load_dataset

data_files = {
    "train": TRAIN_SPLIT_TSV,
    "validation": DEV_SPLIT_TSV,
}

raw_datasets = load_dataset(
    "csv",
    data_files=data_files,
    delimiter="\t",
)

raw_datasets


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['src', 'tgt'],
        num_rows: 475000
    })
    validation: Dataset({
        features: ['src', 'tgt'],
        num_rows: 25000
    })
})

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model_name = "Helsinki-NLP/opus-mt-en-vi"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

source.spm:   0%|          | 0.00/809k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/756k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/289M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
max_source_length = 128
max_target_length = 128

def preprocess_function(examples):
    inputs = examples["src"]
    targets = examples["tgt"]

    model_inputs = tokenizer(
        inputs,
        max_length=max_source_length,
        truncation=True,
        padding=False,   # để collator pad
    )

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets,
            max_length=max_target_length,
            truncation=True,
            padding=False,
        )["input_ids"]

    model_inputs["labels"] = labels
    return model_inputs

tokenized_datasets = raw_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

tokenized_datasets


Map:   0%|          | 0/475000 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4169: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


model.safetensors:   0%|          | 0.00/289M [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 475000
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 25000
    })
})

In [ ]:
from datasets import DatasetDict

max_train_examples = 10_000
max_dev_examples = 1_000

small_train = tokenized_datasets["train"].select(range(max_train_examples))
small_dev = tokenized_datasets["validation"].select(range(max_dev_examples))

small_datasets = DatasetDict({
    "train": small_train,
    "validation": small_dev,
})

print("Train size:", len(small_datasets["train"]))
print("Dev size:", len(small_datasets["validation"]))


Train size: 10000
Dev size: 1000


In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=8,
)

import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup

batch_size = 8         # với model nhỏ, bs=8 thường vẫn ổn
num_epochs = 10

train_loader = DataLoader(
    small_datasets["train"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

dev_loader = DataLoader(
    small_datasets["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)

optimizer = AdamW(model.parameters(), lr=3e-5, weight_decay=0.01)

num_training_steps = num_epochs * len(train_loader)
num_warmup_steps = int(0.1 * num_training_steps)

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=num_warmup_steps,
    num_training_steps=num_training_steps,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


MarianMTModel(
  (model): MarianModel(
    (shared): Embedding(53685, 512, padding_idx=53684)
    (encoder): MarianEncoder(
      (embed_tokens): Embedding(53685, 512, padding_idx=53684)
      (embed_positions): MarianSinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0-5): 6 x MarianEncoderLayer(
          (self_attn): MarianAttention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (activation_fn): SiLU()
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05

In [ ]:
import evaluate
import numpy as np

bleu_metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [p.strip() for p in preds]
    labels = [[l.strip()] for l in labels]
    return preds, labels

def compute_metrics(eval_pred):
    preds, labels = eval_pred

    preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    preds, labels = postprocess_text(preds, labels)
    result = bleu_metric.compute(predictions=preds, references=labels)
    return {"bleu": result["score"]}


In [ ]:
import math
from tqdm.auto import tqdm

best_bleu = -1.0
train_losses = []
train_ppls = []
dev_bleus = []

for epoch in range(num_epochs):
    print(f"\n======== Epoch {epoch+1}/{num_epochs} ========")
    model.train()
    total_train_loss = 0.0
    total_train_tokens = 0

    # TRAIN
    for batch in tqdm(train_loader, desc="Training"):
        batch = {k: v.to(device) for k, v in batch.items()}
        optimizer.zero_grad()

        outputs = model(**batch)
        loss = outputs.loss

        labels = batch["labels"]
        non_pad = (labels != -100).sum().item()

        loss.backward()
        optimizer.step()
        scheduler.step()

        total_train_loss += loss.item() * non_pad
        total_train_tokens += non_pad

    avg_train_loss = total_train_loss / total_train_tokens
    train_ppl = math.exp(avg_train_loss)
    print(f"Train loss: {avg_train_loss:.4f} | Train PPL: {train_ppl:.2f}")
    train_losses.append(avg_train_loss)
    train_ppls.append(train_ppl)

    # EVAL
    model.eval()
    pred_texts = []
    ref_texts = []

    with torch.no_grad():
        for batch in tqdm(dev_loader, desc="Evaluating"):
            labels = batch["labels"]

            gen_batch = {
                "input_ids": batch["input_ids"].to(device),
                "attention_mask": batch["attention_mask"].to(device),
            }

            generated_tokens = model.generate(
                **gen_batch,
                max_length=128,
                num_beams=5,
            )

            # decode pred
            batch_preds = tokenizer.batch_decode(
                generated_tokens,
                skip_special_tokens=True
            )
            pred_texts.extend([p.strip() for p in batch_preds])

            # decode ref
            label_ids = labels.numpy()
            label_ids = np.where(
                label_ids != -100,
                label_ids,
                tokenizer.pad_token_id,
            )
            batch_refs = tokenizer.batch_decode(
                label_ids,
                skip_special_tokens=True
            )
            ref_texts.extend([r.strip() for r in batch_refs])

    print("len preds:", len(pred_texts), "len refs:", len(ref_texts))
    assert len(pred_texts) == len(ref_texts)

    refs_for_metric = [[r] for r in ref_texts]

    bleu_res = bleu_metric.compute(
        predictions=pred_texts,
        references=refs_for_metric,
    )
    bleu = bleu_res["score"]
    print(f"Dev BLEU: {bleu:.2f}")
    dev_bleus.append(bleu)

    if bleu > best_bleu:
        best_bleu = bleu
        print(f"===> New best BLEU: {best_bleu:.2f}, saving model...")
        model.save_pretrained(MODEL_DIR)
        tokenizer.save_pretrained(MODEL_DIR)

print("Best Dev BLEU:", best_bleu)
print("Best model saved at:", MODEL_DIR)


In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import os

# load best model
ft_tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)
ft_model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_DIR).to(device)
ft_model.eval()

# đọc test EN
with open(PUBLIC_TEST_EN, encoding="utf-8") as f:
    test_en_lines = [l.strip() for l in f.readlines()]

with open(PUBLIC_TEST_VI, encoding="utf-8") as f:
    test_vi_lines = [l.strip() for l in f.readlines()]

print("Số câu test:", len(test_en_lines))
print("Ví dụ EN:", test_en_lines[0])

# dịch theo batch
from tqdm.auto import tqdm

batch_size = 32
hypotheses = []

for i in tqdm(range(0, len(test_en_lines), batch_size), desc="Translating test"):
    batch_src = test_en_lines[i : i + batch_size]

    inputs = ft_tokenizer(
        batch_src,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128,
    ).to(device)

    with torch.no_grad():
        output_ids = ft_model.generate(
            **inputs,
            max_length=128,
            num_beams=5,
        )

    batch_hyp = ft_tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    hypotheses.extend([h.strip() for h in batch_hyp])

print("Số câu dịch:", len(hypotheses))
print("Ví dụ HYP:", hypotheses[0])


In [ ]:
src = input("Nhập câu tiếng Anh cần dịch: ")
inputs = ft_tokenizer(src, return_tensors="pt", truncation=True, max_length=128).to(device)

with torch.no_grad():
    output_ids = ft_model.generate(
        **inputs,
        max_length=128,
        num_beams=5,
    )

print(ft_tokenizer.decode(output_ids[0], skip_special_tokens=True))

In [ ]:
import pandas as pd

len_src = len(test_en_lines)
len_hyp = len(hypotheses)
print("len_src:", len_src, "len_hyp:", len_hyp)

min_len = min(len_src, len_hyp)

data = {
    "src_en": test_en_lines[:min_len],
    "hyp_vi": hypotheses[:min_len],
}

# nếu có ref_vi
if 'test_vi_lines' in globals():
    len_ref = len(test_vi_lines)
    print("len_ref:", len_ref)
    min_len = min(min_len, len_ref)
    data = {
        "src_en": test_en_lines[:min_len],
        "hyp_vi": hypotheses[:min_len],
        "ref_vi": test_vi_lines[:min_len],
    }

df_test_res = pd.DataFrame(data)

df_test_res.to_csv(RESULTS_TSV, sep="\t", index=False)

print("Đã lưu:", RESULTS_TSV, "với số dòng:", len(df_test_res))
df_test_res.head()


Biểu đồ Loss, Perplexity, BLEU

In [ ]:
import matplotlib.pyplot as plt

epochs = range(1, len(train_losses) + 1)

plt.figure(figsize=(12, 5))

# Loss
plt.subplot(1, 2, 1)
plt.plot(epochs, train_losses, marker="o")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss")
plt.grid(True)

# Perplexity
plt.subplot(1, 2, 2)
plt.plot(epochs, train_ppls, marker="o", color="orange")
plt.xlabel("Epoch")
plt.ylabel("Perplexity")
plt.title("Training Perplexity")
plt.grid(True)

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(6,4))
plt.plot(epochs, dev_bleus, marker="o", color="green")
plt.xlabel("Epoch")
plt.ylabel("BLEU")
plt.title("Dev BLEU Score")
plt.grid(True)
plt.show()


Đánh giá lỗi

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import sacrebleu

# 1. Tính điểm sentence-level (dùng chrF cho tiếng Việt ổn định hơn)
df_test_res['chrf'] = df_test_res.apply(lambda x: sacrebleu.sentence_chrf(x['hyp_vi'], [x['ref_vi']]).score, axis=1)

# 2. Tính độ dài và tỷ lệ
df_test_res['src_len'] = df_test_res['src_en'].str.split().str.len()
df_test_res['hyp_len'] = df_test_res['hyp_vi'].str.split().str.len()
df_test_res['ratio'] = df_test_res['hyp_len'] / df_test_res['src_len']

# 3. Phân loại lỗi dựa trên ngưỡng (Threshold)
def categorize_error(row):
    if row['chrf'] > 60: return 'Good'
    if row['ratio'] < 0.6: return 'Omission (Dịch thiếu)'
    if row['ratio'] > 1.5: return 'Hallucination (Bịa chữ)'
    return 'Poor Translation (Sai nghĩa/Ngữ pháp)'

df_test_res['error_type'] = df_test_res.apply(categorize_error, axis=1)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 1. Định nghĩa bảng ánh xạ sang tiếng Anh
error_mapping = {
    'Dịch tốt': 'Good Translation',
    'Dịch thiếu (Omission)': 'Omission',
    'Dịch thừa (Hallucination)': 'Hallucination',
    'Sai nặng (Wrong Meaning)': 'Critical Error',
    'Lỗi ngữ pháp/Từ vựng': 'Linguistic Error'
}

# 2. Chuẩn bị dữ liệu
error_counts = df_test_res['error_type'].value_counts()
labels = [error_mapping.get(x, x) for x in error_counts.index]
colors = sns.color_palette('pastel')[0:len(labels)]

# 3. Vẽ biểu đồ
plt.figure(figsize=(10, 8))
plt.pie(
    error_counts,
    labels=labels,
    autopct='%1.1f%%',
    startangle=140,
    colors=colors,
    pctdistance=0.85, # Khoảng cách của chữ số %
    explode=[0.05] * len(labels) # Tạo khoảng cách nhỏ giữa các miếng
)

# Vẽ một hình tròn trắng ở giữa để biến nó thành Donut Chart (nhìn hiện đại hơn)
centre_circle = plt.Circle((0,0), 0.70, fc='white')
fig = plt.gcf()
fig.gca().add_artist(centre_circle)

plt.title('Machine Translation Error Type Distribution', fontsize=15, fontweight='bold', pad=20)
plt.axis('equal')
plt.tight_layout()
plt.show()

In [ ]:
print("--- BÁO CÁO THỐNG KÊ LỖI ---")
summary = df_test_res.groupby('error_type').agg({
    'chrf': 'mean',
    'src_len': 'count'
}).rename(columns={'src_len': 'count', 'chrf': 'avg_chrf'})
print(summary)